In [95]:
from joblib import load
import librosa
import numpy as np

model = load("model0_with_best_params.joblib")
pca = load("pca_transform.joblib")


In [96]:
def extract_mfcc(file_path):
    y, sr = librosa.load(file_path, sr=16000)
    y, _ = librosa.effects.trim(y)
    y = librosa.util.normalize(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)

    if mfcc.shape[1] >= 100:
        mfcc = mfcc[:, :100]
    else:
        pad_width = 100 - mfcc.shape[1]
        mfcc = np.pad(mfcc, ((0, 0), (0, pad_width)), mode='constant')

    return mfcc


In [ ]:
def extract_true_label(filename):
    if '_' in filename:
        emotion = filename.split('_')[2].split('.')[0].lower()
        
        return  emotion
    elif '-' in filename:
        code = filename.split('-')[2]
        emotion_map = {
            '01': 'neutral',
            '02': 'calm',
            '03': 'happy', 
            '04': 'sad',
            '05': 'angry',
            '06': 'fear',
            '07': 'disgust',
            '08': 'surprised'
        }
        return emotion_map.get(code)

In [98]:
from sklearn.preprocessing import LabelEncoder


def get_label_encoder():
    duygular = [
    'angry',
    'calm',
    'disgust',
    'fear',
    'fearful',     
    'happy',
    'neutral',
    'ps',          
    'sad',
    'surprised'
]
    encoder = LabelEncoder()
    encoder.fit(duygular)
    return encoder

In [101]:
import os
from sklearn.metrics import classification_report


def predict_and_evaluate_mlp(folder_path, model, pca, label_encoder):
    files = [f for f in os.listdir(folder_path) if f.endswith('.wav')]

    X = []
    y_true = []

    for file in files:
        file_path = os.path.join(folder_path, file)
        label = extract_true_label(file)

        mfcc = extract_mfcc(file_path)
        X_flat = mfcc.reshape(1, -1)
        X_reduced = pca.transform(X_flat) 

        X.append(X_reduced[0])
        y_true.append(label)


    X = np.array(X)
    y_true_encoded = label_encoder.transform(y_true)
    y_pred = model.predict(X)

    labels = np.unique(y_true_encoded)
    target_names = label_encoder.inverse_transform(labels)
    print(y_true_encoded)
    print(y_pred)
    print(np.unique(y_pred))
    print(label_encoder.classes_)
    print("İlk 10 Dosya İçin:")
    print(classification_report(y_true_encoded, y_pred, target_names=target_names))


In [102]:
encoder = get_label_encoder()
predict_and_evaluate_mlp("ses/test_10",model,pca,encoder)

[1 8 9 3 2 2 7 8 2 6]
[1 8 9 3 2 2 7 8 2 6]
[1 2 3 6 7 8 9]
['angry' 'calm' 'disgust' 'fear' 'fearful' 'happy' 'neutral' 'ps' 'sad'
 'surprised']
İlk 10 Dosya İçin:
              precision    recall  f1-score   support

        calm       1.00      1.00      1.00         1
     disgust       1.00      1.00      1.00         3
        fear       1.00      1.00      1.00         1
     neutral       1.00      1.00      1.00         1
          ps       1.00      1.00      1.00         1
         sad       1.00      1.00      1.00         2
   surprised       1.00      1.00      1.00         1

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10

